<a href="https://colab.research.google.com/github/bhartiansh/cnn_pruning_cifar10/blob/main/SNIP_ResNet20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/bhartiansh/cnn_pruning_cifar10.git
%cd cnn_pruning_cifar10

!pip install -q tensorflow-model-optimization

Cloning into 'cnn_pruning_cifar10'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 212 (delta 36), reused 2 (delta 2), pack-reused 144 (from 1)
Receiving objects: 100% (212/212), 130.56 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/cnn_pruning_cifar10


In [ ]:
import tensorflow as tf
import numpy as np
from models.resnet20 import build_resnet20

def compute_snip_scores(model, x_sample, y_sample):
    with tf.GradientTape() as tape:
        logits = model(x_sample, training=True)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_sample, logits)
        loss = tf.reduce_mean(loss)

    grads = tape.gradient(loss, model.trainable_weights)
    scores = []

    for w, g in zip(model.trainable_weights, grads):
        if 'kernel' in w.name:
            scores.append(tf.reshape(tf.abs(w * g), [-1]))

    return tf.concat(scores, axis=0)

def apply_snip_mask(model, x_sample, y_sample, sparsity=0.5):
    scores = compute_snip_scores(model, x_sample, y_sample)
    k = int((1 - sparsity) * scores.shape[0])
    threshold = tf.sort(scores)[k]

    for w in model.trainable_weights:
        if 'kernel' in w.name:
            mask = tf.cast(tf.abs(w) >= threshold, tf.float32)
            w.assign(w * mask)

def quick_train(model, x_train, y_train, x_test, y_test, epochs=10, batch_size=64):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=batch_size,
              epochs=epochs,
              verbose=2)
    return model

In [ ]:
# === Run it ===
if __name__ == "__main__":
    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()

    # Build model
    model = build_resnet20()

    # Run dummy forward pass to initialize weights
    model(tf.convert_to_tensor(x_train[:1]), training=False)

    # SNIP pruning using small batch
    apply_snip_mask(model, x_train[:64], y_train[:64], sparsity=0.3)

    # Very quick fine-tuning
    quick_train(model, x_train, y_train, x_test, y_test, epochs=30, batch_size=64)

170498071/170498071 [==============================] - 4s 0us/step
Epoch 1/30
782/782 - 61s - loss: 1.3401 - accuracy: 0.5109 - val_loss: 1.1363 - val_accuracy: 0.5894 - 61s/epoch - 78ms/step
Epoch 2/30
782/782 - 54s - loss: 0.9124 - accuracy: 0.6764 - val_loss: 1.2805 - val_accuracy: 0.5745 - 54s/epoch - 69ms/step
Epoch 3/30
782/782 - 57s - loss: 0.7387 - accuracy: 0.7416 - val_loss: 1.0174 - val_accuracy: 0.6621 - 57s/epoch - 73ms/step
Epoch 4/30
782/782 - 57s - loss: 0.6244 - accuracy: 0.7817 - val_loss: 0.7847 - val_accuracy: 0.7267 - 57s/epoch - 74ms/step
Epoch 5/30
782/782 - 55s - loss: 0.5460 - accuracy: 0.8086 - val_loss: 1.1357 - val_accuracy: 0.6390 - 55s/epoch - 70ms/step
Epoch 6/30
782/782 - 54s - loss: 0.4848 - accuracy: 0.8329 - val_loss: 0.7083 - val_accuracy: 0.7633 - 54s/epoch - 69ms/step
Epoch 7/30
782/782 - 54s - loss: 0.4294 - accuracy: 0.8519 - val_loss: 0.7287 - val_accuracy: 0.7584 - 54s/epoch - 69ms/step
Epoch 8/30
782/782 - 54s - loss: 0.3783 - accuracy: 0.8685

In [ ]:
# === Run it ===
if __name__ == "__main__":
    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()

    # Build model
    model = build_resnet20()

    # Run dummy forward pass to initialize weights
    model(tf.convert_to_tensor(x_train[:1]), training=False)

    # SNIP pruning using small batch
    apply_snip_mask(model, x_train[:64], y_train[:64], sparsity=0.5)

    # Very quick fine-tuning
    quick_train(model, x_train, y_train, x_test, y_test, epochs=30, batch_size=64)

170498071/170498071 [==============================] - 11s 0us/step
Epoch 1/30
782/782 - 347s - loss: 1.3104 - accuracy: 0.5265 - val_loss: 1.8690 - val_accuracy: 0.4699 - 347s/epoch - 444ms/step
Epoch 2/30
782/782 - 342s - loss: 0.8921 - accuracy: 0.6839 - val_loss: 0.9585 - val_accuracy: 0.6720 - 342s/epoch - 437ms/step
Epoch 3/30
782/782 - 341s - loss: 0.7132 - accuracy: 0.7499 - val_loss: 0.8214 - val_accuracy: 0.7183 - 341s/epoch - 436ms/step
Epoch 4/30
782/782 - 335s - loss: 0.6087 - accuracy: 0.7879 - val_loss: 0.8776 - val_accuracy: 0.7090 - 335s/epoch - 429ms/step
Epoch 5/30
782/782 - 337s - loss: 0.5343 - accuracy: 0.8143 - val_loss: 0.7388 - val_accuracy: 0.7482 - 337s/epoch - 431ms/step
Epoch 6/30
782/782 - 333s - loss: 0.4738 - accuracy: 0.8357 - val_loss: 0.7324 - val_accuracy: 0.7598 - 333s/epoch - 426ms/step
Epoch 7/30
782/782 - 335s - loss: 0.4178 - accuracy: 0.8543 - val_loss: 0.8825 - val_accuracy: 0.7353 - 335s/epoch - 428ms/step
Epoch 8/30
782/782 - 334s - loss: 0.

In [ ]:
# === Run it ===
if __name__ == "__main__":
    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()

    # Build model
    model = build_resnet20()

    # Run dummy forward pass to initialize weights
    model(tf.convert_to_tensor(x_train[:1]), training=False)

    # SNIP pruning using small batch
    apply_snip_mask(model, x_train[:64], y_train[:64], sparsity=0.7)

    # Very quick fine-tuning
    quick_train(model, x_train, y_train, x_test, y_test, epochs=30, batch_size=64)

Epoch 1/30
782/782 - 344s - loss: 1.3406 - accuracy: 0.5134 - val_loss: 1.9493 - val_accuracy: 0.4587 - 344s/epoch - 440ms/step
Epoch 2/30
782/782 - 339s - loss: 0.9146 - accuracy: 0.6769 - val_loss: 1.2685 - val_accuracy: 0.5858 - 339s/epoch - 433ms/step
Epoch 3/30
782/782 - 335s - loss: 0.7333 - accuracy: 0.7429 - val_loss: 1.1805 - val_accuracy: 0.6415 - 335s/epoch - 429ms/step
Epoch 4/30
782/782 - 335s - loss: 0.6250 - accuracy: 0.7823 - val_loss: 0.8109 - val_accuracy: 0.7253 - 335s/epoch - 429ms/step
Epoch 5/30
782/782 - 337s - loss: 0.5442 - accuracy: 0.8109 - val_loss: 0.7143 - val_accuracy: 0.7580 - 337s/epoch - 430ms/step
Epoch 6/30
782/782 - 337s - loss: 0.4838 - accuracy: 0.8318 - val_loss: 0.9642 - val_accuracy: 0.7066 - 337s/epoch - 430ms/step
Epoch 7/30
782/782 - 335s - loss: 0.4270 - accuracy: 0.8509 - val_loss: 0.7065 - val_accuracy: 0.7658 - 335s/epoch - 428ms/step
Epoch 8/30
782/782 - 341s - loss: 0.3816 - accuracy: 0.8663 - val_loss: 0.7525 - val_accuracy: 0.7628 - 

In [ ]:
# 1. Force compatible versions (reset everything to Colab defaults)
!pip install -U --force-reinstall numpy==1.23.5
!pip install -U --force-reinstall tensorflow==2.14.0
!pip install -U tensorflow-model-optimization

# 2. Restart runtime automatically after install
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 118.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.2.2
    Uninstalling absl-py-2.2.2:
      Successfully uninstalled absl-py-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.6 which is incompatible.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.2.0 which is incompatible.
tensorflow-deci